In [21]:
#Import the major packages for data pulling

import pandas as pd
import requests
import time
from datetime import datetime

In [22]:
#Saving the json urls for the Anime subreddit posts

url_anime = 'https://www.reddit.com/r/anime.json'

#This is like a soft login to tell reddit who you are so they can keep track of you.
#This is us vouching that we aren't a robot or a malicious entity. Now we have access
header = {'User-agent' : 'DSI demo'}  

In [23]:
#Same as above, but for the Anime subreddit server
res_anime = requests.get(url_anime, headers = header)
res_anime

<Response [200]>

In [24]:
#Converts and assigns json document to something we can perform python on
anime_json = res_anime.json()

In [25]:
anime_json

{'kind': 'Listing',
 'data': {'modhash': '',
  'dist': 27,
  'children': [{'kind': 't3',
    'data': {'approved_at_utc': None,
     'subreddit': 'anime',
     'selftext': "\nThis is a weekly thread to get to know /r/anime's community. Talk about your day-to-day life, share your hobbies, or make small talk with your fellow anime fans.\n\nAlthough this is a place for off-topic discussion, there are a few rules to keep in mind:\n\n1. Be courteous and respectful of other users.\n\n2. Discussion of religion, politics, depression, and other similar topics will be moderated due to their sensitive nature. While we encourage users to talk about their daily lives and get to know others, this thread is not intended for extended discussion of the aforementioned topics or for emotional support.\n\n3. Roleplaying is not allowed. This behaviour is not appropriate as it is obtrusive to uninvolved users.\n\n4. No meta discussion. If you have a meta concern, please raise it in the Monthly Meta Thread an

A .json document can be parsed through like a python dictionary. I'm going to try to find the posts for each subreddit.

In [26]:
anime_json['data']['children'][0]['data']['title']

'Casual Discussion Fridays - Week of July 12, 2019'

Now that I've located the information I want to extract from the post, I can move on to making a list of dictionaries that can be converted into a pandas dataframe to store this information into rows and columns.

In [27]:
#each is saying for the first post, second post, third post, make a dataframe where the keys are the columns 
#and the rows are the values.
pd.DataFrame([each['data'] for each in anime_json['data']['children']])

,all_awardings,allow_live_comments,approved_at_utc,approved_by,archived,author,author_flair_background_color,author_flair_css_class,author_flair_richtext,author_flair_template_id,...,thumbnail_width,title,total_awards_received,ups,url,user_reports,view_count,visited,whitelist_status,wls
0,[],True,None,None,False,AnimeMod,None,None,[],None,...,NaN,"Casual Discussion Fridays - Week of July 12, 2019",0,63,https://www.reddit.com/r/anime/comments/cc3e6e...,[],None,False,all_ads,6
1,[],False,None,None,False,pittman66,,MAL,[],None,...,NaN,"Wiki Overhaul Month, Week 3.33: FAQ (Frequentl...",0,43,https://www.reddit.com/r/anime/comments/cc3e0b...,[],None,False,all_ads,6
2,[],True,None,None,False,AutoLovepon,None,None,[],None,...,NaN,Dr. Stone - Episode 2 discussion,0,2311,https://www.reddit.com/r/anime/comments/ccbm1g...,[],None,False,all_ads,6
3,[],False,None,None,False,AutoLovepon,None,None,[],None,...,NaN,Enen no Shouboutai - Episode 2 discussion,0,1240,https://www.reddit.com/r/anime/comments/ccdtpe...,[],None,False,all_ads,6
4,"[{'is_enabled': True, 'count': 1, 'subreddit_i...",True,None,None,False,Max_58,None,None,[],None,...,140.0,Neon Genesis Evangelion Unit 01 and 02,1,6480,https://i.redd.it/e8v68dgwgu931.png,[],None,False,all_ads,6
5,[],False,None,None,False,AutoLovepon,None,None,[],None,...,NaN,Dungeon ni Deai o Motomeru no wa Machigatte Ir...,0,1143,https://www.reddit.com/r/anime/comments/ccd4wt...,[],None,False,all_ads,6
6,"[{'is_enabled': True, 'count': 1, 'subreddit_i...",True,None,None,False,AutoLovepon,None,None,[],None,...,NaN,Tsuujou Kougeki ga Zentai Kougeki de Ni-kai Ko...,1,533,https://www.reddit.com/r/anime/comments/ccdpqi...,[],None,False,all_ads,6
7,[],True,None,None,False,Master-Samwell,None,None,[],None,...,140.0,To this day I’m still not sure how I feel abou...,0,971,https://v.redd.it/frf1xleafv931,[],None,False,all_ads,6
8,[],True,None,None,False,AutoLovepon,None,None,[],None,...,NaN,Fruits Basket - Episode 15 discussion,0,405,https://www.reddit.com/r/anime/comments/ccdfcf...,[],None,False,all_ads,6
9,[],True,None,None,False,AutoLovepon,None,None,[],None,...,NaN,Araburu Kisetsu no Otome-domo yo. - Episode 2 ...,0,231,https://www.reddit.com/r/anime/comments/ccemqu...,[],None,False,all_ads,6


Now that I've created a dataframe from my json, I'd like to design a function that does this for me programmatically.

In [28]:
def json_to_df(reddit_json):
    df = pd.DataFrame([each['data'] for each in reddit_json['data']['children']])
    return df

In [29]:
after = {}
list_of_dfs = []

for subreddit_page in range(40):
    res_anime = requests.get(url_anime, headers = header, params = after)
    anime_json = res_anime.json()
    list_of_dfs.append(json_to_df(anime_json))
    after = {'after': anime_json['data']['after']}
    time.sleep(3)

In [30]:
anime_posts = pd.concat(list_of_dfs, sort = False, ignore_index= True)  #Without the ignore index param, our indices will repeat
len(set(anime_posts['title']))

837

In [32]:
dt = str(datetime.now()).split('.')[0].replace(':', '_')
subreddit = 'Anime'
anime_posts.to_csv(f'../data/Anime_{dt}.csv', index = False)